### 00. 라이브러리 설치

In [2]:
!pip install langchain
!pip install langchain-community
!pip install langchain-openai
!pip install pypdf
!pip install chromadb
!pip install tiktoken


[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


### 01. 라이브러리 임포트 / 환경 설정

In [3]:
import os
from pathlib import Path
from typing import List

# LangChain 핵심 모듈
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import (
    RecursiveCharacterTextSplitter,
    CharacterTextSplitter,
)
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.schema import Document

# 환경 변수 설정
import warnings

warnings.filterwarnings("ignore")

In [ ]:
# OpenAI API 키 설정
os.environ["OPENAI_API_KEY"] = "sk-proj..."

In [5]:
# 파일 경로 설정
PDF_PATH = r"D:\data"  # Windows 경로
PERSIST_DIRECTORY = r"D:\data\chroma_db"  # 벡터 DB 저장 경로

### 02. PDF 문서 로드

In [6]:
def load_pdf_documents(directory_path: str) -> List[Document]:
    """
    지정된 디렉토리에서 모든 PDF 파일을 로드
    """
    print(f"{directory_path}에서 PDF 파일 로드 중...")

    # 방법 1: 개별 PDF 파일 로드
    pdf_files = list(Path(directory_path).glob("*.pdf"))

    if not pdf_files:
        print(" * PDF 파일을 찾을 수 없습니다!")
        return []

    all_docs = []
    for pdf_file in pdf_files[:2]:  # 2개만 로드
        print(f" * 로딩: {pdf_file.name}")
        loader = PyPDFLoader(str(pdf_file))
        docs = loader.load()

        # 메타데이터 추가
        for doc in docs:
            doc.metadata["source"] = pdf_file.name
            doc.metadata["file_path"] = str(pdf_file)

        all_docs.extend(docs)

    print(f" * 총 {len(all_docs)}개 페이지 로드 완료")
    return all_docs


# 문서 로드 실행
documents = load_pdf_documents(PDF_PATH)

# 로드된 문서 확인
print(f"\n * [ 로드된 문서 정보 ]")
print(f"  - 총 페이지 수: {len(documents)}")
print(f"  - 첫 번째 페이지 미리보기: {documents[0].page_content[:200]}...")

D:\data에서 PDF 파일 로드 중...
 * 로딩: 2023 당뇨병 진료지침_전문_240620.pdf
 * 총 428개 페이지 로드 완료

 * [ 로드된 문서 정보 ]
  - 총 페이지 수: 428
  - 첫 번째 페이지 미리보기: 1
Clinical Practice Guidelines for Diabetes 
당
뇨
병 
진
료
지
침

제             판
당뇨병 진단 및 분류
당뇨병 선별검사
임신당뇨병 선별과 진단
2형당뇨병의 예방
당뇨병 성인에게서 혈당조절 목표
혈당조절의 모니터링 및 평가
당뇨병 자기관리
의학영양요법
운동요법
1형당뇨병의 약물치료
2형당뇨병의 ...


### 03. 텍스트 분할 (Chunking) - 2가지 방법 비교

In [7]:
# 방법 1: RecursiveCharacterTextSplitter
print("\n * 텍스트 분할 방법 1: RecursiveCharacterTextSplitter")
recursive_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,  # 청크 크기
    chunk_overlap=200,  # 청크 간 중복
    length_function=len,
    separators=["\n\n", "\n", " ", ""],  # 우선순위별 구분자
)

recursive_chunks = recursive_splitter.split_documents(documents)
print(f"  - 생성된 청크 수: {len(recursive_chunks)}")
print(
    f"  - 평균 청크 크기: {sum(len(c.page_content) for c in recursive_chunks) / len(recursive_chunks):.0f}자"
)

# 방법 2: CharacterTextSplitter
print("\n * 텍스트 분할 방법 2: CharacterTextSplitter")
character_splitter = CharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200, separator="\n"  # 단일 구분자
)

character_chunks = character_splitter.split_documents(documents)
print(f"  - 생성된 청크 수: {len(character_chunks)}")
print(
    f"  - 평균 청크 크기: {sum(len(c.page_content) for c in character_chunks) / len(character_chunks):.0f}자"
)

# 비교를 위해 recursive_chunks 사용
chunks = recursive_chunks


 * 텍스트 분할 방법 1: RecursiveCharacterTextSplitter
  - 생성된 청크 수: 925
  - 평균 청크 크기: 742자

 * 텍스트 분할 방법 2: CharacterTextSplitter
  - 생성된 청크 수: 927
  - 평균 청크 크기: 741자


### 04. 임베딩 생성 및 벡터 DB 저장

In [8]:
print("\n* 임베딩 생성 및 벡터 DB 구축 중...")

# OpenAI 임베딩 모델 초기화
embeddings = OpenAIEmbeddings(
    model="text-embedding-ada-002", chunk_size=1000  # 한 번에 처리할 청크 개수 제한
)

# 방법 1: 배치 처리로 나누어 저장
from tqdm import tqdm  # 진행 상황 표시용


def create_vectorstore_in_batches(chunks, batch_size=100):
    """
    큰 문서를 배치로 나누어 벡터 DB 생성
    """
    vectorstore = None
    total_chunks = len(chunks)

    print(f"총 {total_chunks}개의 청크를 {batch_size}개씩 처리합니다...")

    for i in range(0, total_chunks, batch_size):
        batch = chunks[i : i + batch_size]
        print(f"처리 중: {i+1}-{min(i+batch_size, total_chunks)} / {total_chunks}")

        if vectorstore is None:
            # 첫 배치: 새 벡터스토어 생성
            vectorstore = Chroma.from_documents(
                documents=batch,
                embedding=embeddings,
                persist_directory=PERSIST_DIRECTORY,
                collection_metadata={"hnsw:space": "cosine"},
            )
        else:
            # 이후 배치: 기존 벡터스토어에 추가
            vectorstore.add_documents(batch)

    return vectorstore


# 배치 처리로 벡터 DB 생성
vectorstore = create_vectorstore_in_batches(chunks, batch_size=50)

# 벡터 DB 저장
vectorstore.persist()
print(f"* 벡터 DB 저장 완료: {PERSIST_DIRECTORY}")


* 임베딩 생성 및 벡터 DB 구축 중...
총 925개의 청크를 50개씩 처리합니다...
처리 중: 1-50 / 925
처리 중: 51-100 / 925
처리 중: 101-150 / 925
처리 중: 151-200 / 925
처리 중: 201-250 / 925
처리 중: 251-300 / 925
처리 중: 301-350 / 925
처리 중: 351-400 / 925
처리 중: 401-450 / 925
처리 중: 451-500 / 925
처리 중: 501-550 / 925
처리 중: 551-600 / 925
처리 중: 601-650 / 925
처리 중: 651-700 / 925
처리 중: 701-750 / 925
처리 중: 751-800 / 925
처리 중: 801-850 / 925
처리 중: 851-900 / 925
처리 중: 901-925 / 925
* 벡터 DB 저장 완료: D:\data\chroma_db


### 05. 검색 테스트

In [9]:
print("\n * 검색 테스트 시작")

# 테스트 질문들
test_queries = [
    "이 문서의 주요 내용은 무엇인가요?",
    "당뇨병의 진단기준에 대해 알려주세요.",
    "당뇨병의 운동요법은 어떤것이 있나요?",
]


# 검색 함수
def search_documents(query: str, k: int = 3):
    """
    질문에 대해 가장 유사한 문서 검색
    """
    print(f"\n* 질문: {query}")
    print("-" * 50)

    # 유사도 검색
    results = vectorstore.similarity_search_with_score(query, k=k)

    for i, (doc, score) in enumerate(results, 1):
        print(f"\n* 결과 {i} (유사도: {score:.3f})")
        print(
            f"   출처: {doc.metadata.get('source', 'Unknown')}, "
            f"페이지: {doc.metadata.get('page', 'N/A')}"
        )
        print(f"   내용: {doc.page_content[:200]}...")

    return results


# 각 질문에 대해 검색 수행
for query in test_queries:
    results = search_documents(query)


 * 검색 테스트 시작

* 질문: 이 문서의 주요 내용은 무엇인가요?
--------------------------------------------------

* 결과 1 (유사도: 0.204)
   출처: 2023 당뇨병 진료지침_전문_240620.pdf, 페이지: 417
   내용: 신설하여 이에 포함시켜 기술하였다. 당뇨병환자의 약제선택 알고리듬, 인슐린치료 알고리듬을 업데이트 하
였고, 고혈압 관리, 이상지질혈증 관리에 대한 알고리듬을 신설하였으며, 당뇨병환자의 포괄적관리를 위한 
점검사항을 표로 정리하였다. 제6판은 부의 구분 없이 총 27개 장으로 구성하였고, 요약하여 영문 종설원고
의 형태로 대한당뇨병학회 학회지에 발표하였다....

* 결과 2 (유사도: 0.204)
   출처: 2023 당뇨병 진료지침_전문_240620.pdf, 페이지: 408
   내용: - 문헌검색은 전문사서 前) 제일병원 의학도서실 이성욱, 강동경희대학교병원 의학도서실 이영진에 의해 체계적으로 
  수행함.
- 각 핵심질문은 2022년 진료지침에 사용했던 검색어, 검색식을 업데이트 및 보강함.
- 29개 세부 소주제의 검색어, 검색식 및 검색결과는 파일 형태로 보관함. 
2) 근거(진료지침)의 검색
●
  근거문헌에 대해 집필그룹의 구...

* 결과 3 (유사도: 0.208)
   출처: 2023 당뇨병 진료지침_전문_240620.pdf, 페이지: 39
   내용: 서 체중, 당화혈색소, 당뇨병 발생 위험 등 주요 평가변수에서 중재군과 대조군 간 비교에서 일관적인 결과
는 확인되지 않았다. 인터넷 또는 모바일 활용 교육프로그램을 보조수단으로 활용한 일부 연구에서 체중이
나 체질량지수 등 주요 임상지표를 일정기간 유의하게 개선시켰다[26,27]. 향후 정보통신기술을 활용한 생
활습관중재의 효과를 지속적으로 유지할 수 있...

* 질문: 당뇨병의 진단기준에 대해 알려주세요.
--------------------------------------------------

### 06. 성능 비교 분석

In [10]:
print("\n * 청킹 방법 성능 비교")


def compare_chunking_methods():
    """
    두 가지 청킹 방법의 성능 비교
    """
    methods = {"Recursive": recursive_chunks, "Character": character_chunks}

    test_query = "이 문서의 핵심 내용은?"

    for method_name, chunks in methods.items():
        # 각 방법으로 벡터 DB 생성
        temp_vectorstore = Chroma.from_documents(
            documents=chunks[:50], embedding=embeddings  # 테스트용으로 50개만
        )

        # 검색 수행
        results = temp_vectorstore.similarity_search_with_score(test_query, k=3)

        print(f"\n* {method_name} Splitter 결과:")
        print(f"   - 총 청크 수: {len(chunks)}")
        print(f"   - 최고 유사도 점수: {results[0][1]:.3f}")
        print(f"   - 검색된 내용 길이: {len(results[0][0].page_content)}자")


compare_chunking_methods()


 * 청킹 방법 성능 비교

* Recursive Splitter 결과:
   - 총 청크 수: 925
   - 최고 유사도 점수: 0.431
   - 검색된 내용 길이: 251자

* Character Splitter 결과:
   - 총 청크 수: 927
   - 최고 유사도 점수: 0.430
   - 검색된 내용 길이: 251자


### 07. 다양한 검색 방법 테스트

In [11]:
print("\n* 고급 검색 기법")

# MMR (Maximal Marginal Relevance) 검색 - 다양성 있는 결과
print("\n1. MMR 검색 (다양성 있는 결과):")
mmr_results = vectorstore.max_marginal_relevance_search(
    query="당뇨병의 진단 방법", k=4, fetch_k=10
)
for i, doc in enumerate(mmr_results, 1):
    print(f"  - 결과 {i}: {doc.page_content[:100]}...")

# 메타데이터 필터링 검색
print("\n2. 메타데이터 필터링 검색:")
if len(documents) > 0:
    first_source = documents[0].metadata.get("source")
    filtered_results = vectorstore.similarity_search(
        query="당뇨병의 진단 방법", k=2, filter={"source": first_source}
    )
    print(f"  '{first_source}' 파일에서만 검색:")
    for doc in filtered_results:
        print(f"    - {doc.page_content[:100]}...")


* 고급 검색 기법

1. MMR 검색 (다양성 있는 결과):
  - 결과 1: 뇨병학 용어집(제4판, 2021)”을 기준으로 용어를 통일하였습니다.
최근 30대 젊은 성인에서 비만의 유병률이 증가하고 있고, 당뇨병전단계에 해당되는 사람이 30% 이상임
을 고...
  - 결과 2: 호전시킬 수 있음이 확인되었다[7,8].
2. 위해
혈당, 혈압, 지질조절에 해당하는 위해는 내과적 약물치료의 잠재적 위해 가능성을 따른다. 
1. 이득
증식당뇨병망막병증이나 황반...
  - 결과 3: 질환 등)을 발생시킬 수 있다.
4. 이득과 위해의 균형
당뇨병신경병증에 사용하는 약물에 대한 효과와 부작용을 이득과 위해로 평가해 보면, 이득이 위해를 상회
하여 당뇨병신경병증 ...
  - 결과 4: 고령층에서 당뇨병은 매우 흔하나, 젊은 성인들에 비해 매우 이질적인 양상을 보인다. 따라서 현재 혈당상태
나, 동반질환, 당뇨병합병증만을 평가해서는 안되고, 포괄적인 노인평가가 이...

2. 메타데이터 필터링 검색:
  '2023 당뇨병 진료지침_전문_240620.pdf' 파일에서만 검색:
    - 뇨병학 용어집(제4판, 2021)”을 기준으로 용어를 통일하였습니다.
최근 30대 젊은 성인에서 비만의 유병률이 증가하고 있고, 당뇨병전단계에 해당되는 사람이 30% 이상임
을 고...
    - 호전시킬 수 있음이 확인되었다[7,8].
2. 위해
혈당, 혈압, 지질조절에 해당하는 위해는 내과적 약물치료의 잠재적 위해 가능성을 따른다. 
1. 이득
증식당뇨병망막병증이나 황반...


### 08. 검색 품질 개선 실험

In [12]:
# 청크 사이즈 실험 및 평가


def experiment_chunk_sizes(documents, test_queries):
    """
    여러 청크 사이즈를 실험하고 최적값 찾기
    """
    chunk_sizes = [500, 1000, 1500]
    overlaps = [50, 100, 200]

    experiment_results = []

    print("청크 사이즈 실험 시작...")

    for chunk_size in chunk_sizes:
        for overlap in overlaps:
            print(f"\n테스트: chunk_size={chunk_size}, overlap={overlap}")

            # 1. 해당 설정으로 청킹
            splitter = RecursiveCharacterTextSplitter(
                chunk_size=chunk_size, chunk_overlap=overlap
            )
            test_chunks = splitter.split_documents(documents)

            # 2. 임시 벡터 DB 생성
            temp_vectorstore = Chroma.from_documents(
                documents=test_chunks,
                embedding=embeddings,
                collection_name=f"test_{chunk_size}_{overlap}",
            )

            # 3. 검색 품질 평가
            total_score = 0
            for query in test_queries:
                results = temp_vectorstore.similarity_search_with_score(query, k=3)

                # 상위 3개 결과의 평균 점수 계산
                avg_score = sum(score for _, score in results) / len(results)
                total_score += avg_score

            avg_total_score = total_score / len(test_queries)

            # 4. 결과 저장
            experiment_results.append(
                {
                    "chunk_size": chunk_size,
                    "overlap": overlap,
                    "num_chunks": len(test_chunks),
                    "avg_score": avg_total_score,
                    "avg_chunk_length": sum(len(c.page_content) for c in test_chunks)
                    / len(test_chunks),
                }
            )

            print(f"  - 청크 개수: {len(test_chunks)}")
            print(f"  - 평균 검색 점수: {avg_total_score:.3f}")

    return experiment_results


# 최적 설정 찾기


def find_best_settings(experiment_results):
    """
    실험 결과에서 최적 설정 찾기
    """
    # 검색 점수가 가장 높은 설정 찾기
    best_by_score = min(experiment_results, key=lambda x: x["avg_score"])

    # 청크 개수와 점수의 균형이 좋은 설정 찾기 (효율성)
    best_balanced = min(
        experiment_results, key=lambda x: x["avg_score"] * (1 + x["num_chunks"] / 1000)
    )

    print("\n=== 실험 결과 분석 ===")
    print(
        f"최고 검색 품질: chunk_size={best_by_score['chunk_size']}, "
        f"overlap={best_by_score['overlap']} (점수: {best_by_score['avg_score']:.3f})"
    )
    print(
        f"균형잡힌 설정: chunk_size={best_balanced['chunk_size']}, "
        f"overlap={best_balanced['overlap']}"
    )

    return best_by_score


# 최적 설정으로 실제 벡터 DB 구축


def build_optimized_vectorstore(documents, best_settings):
    """
    최적 설정으로 최종 벡터 DB 구축
    """
    print(f"\n최적 설정으로 벡터 DB 구축 중...")
    print(
        f"선택된 설정: chunk_size={best_settings['chunk_size']}, "
        f"overlap={best_settings['overlap']}"
    )

    # 최적 설정으로 스플리터 생성
    optimal_splitter = RecursiveCharacterTextSplitter(
        chunk_size=best_settings["chunk_size"], chunk_overlap=best_settings["overlap"]
    )

    # 최종 청킹
    final_chunks = optimal_splitter.split_documents(documents)
    print(f"최종 청크 개수: {len(final_chunks)}")

    # 최종 벡터 DB 생성
    final_vectorstore = Chroma.from_documents(
        documents=final_chunks,
        embedding=embeddings,
        persist_directory=PERSIST_DIRECTORY,
        collection_metadata={
            "chunk_size": best_settings["chunk_size"],
            "overlap": best_settings["overlap"],
            "optimized_for": "search_quality",
        },
    )

    final_vectorstore.persist()
    return final_vectorstore

In [13]:
# 전체 프로세스 실행

# Step 1: 실험 실행
test_queries = [
    "이 문서의 주요 내용은 무엇인가요?",
    "유방암의 병리학적 평가에 대해 알려주세요.",
    "아로마타제억제제가 무엇인가요?",
]

results = experiment_chunk_sizes(documents[:10], test_queries)  # 일부 문서로 테스트

# Step 2: 최적 설정 찾기
best = find_best_settings(results)

# Step 3: 최적 설정 적용
final_vectorstore = build_optimized_vectorstore(documents, best)

# Step 4: 개선 효과 검증
print("\n=== 개선 효과 검증 ===")
for query in test_queries:
    results_before = vectorstore.similarity_search_with_score(query, k=1)  # 이전
    results_after = final_vectorstore.similarity_search_with_score(query, k=1)  # 이후

    print(f"\n질문: {query}")
    print(f"  개선 전 점수: {results_before[0][1]:.3f}")
    print(f"  개선 후 점수: {results_after[0][1]:.3f}")
    print(
        f"  개선율: {((results_before[0][1] - results_after[0][1]) / results_before[0][1] * 100):.1f}%"
    )

청크 사이즈 실험 시작...

테스트: chunk_size=500, overlap=50
  - 청크 개수: 21
  - 평균 검색 점수: 0.397

테스트: chunk_size=500, overlap=100
  - 청크 개수: 21
  - 평균 검색 점수: 0.394

테스트: chunk_size=500, overlap=200
  - 청크 개수: 25
  - 평균 검색 점수: 0.396

테스트: chunk_size=1000, overlap=50
  - 청크 개수: 14
  - 평균 검색 점수: 0.416

테스트: chunk_size=1000, overlap=100
  - 청크 개수: 14
  - 평균 검색 점수: 0.413

테스트: chunk_size=1000, overlap=200
  - 청크 개수: 14
  - 평균 검색 점수: 0.414

테스트: chunk_size=1500, overlap=50
  - 청크 개수: 10
  - 평균 검색 점수: 0.441

테스트: chunk_size=1500, overlap=100
  - 청크 개수: 10
  - 평균 검색 점수: 0.441

테스트: chunk_size=1500, overlap=200
  - 청크 개수: 10
  - 평균 검색 점수: 0.441

=== 실험 결과 분석 ===
최고 검색 품질: chunk_size=500, overlap=100 (점수: 0.394)
균형잡힌 설정: chunk_size=500, overlap=100

최적 설정으로 벡터 DB 구축 중...
선택된 설정: chunk_size=500, overlap=100
최종 청크 개수: 1627

=== 개선 효과 검증 ===

질문: 이 문서의 주요 내용은 무엇인가요?
  개선 전 점수: 0.175
  개선 후 점수: 0.175
  개선율: 0.0%

질문: 유방암의 병리학적 평가에 대해 알려주세요.
  개선 전 점수: 0.134
  개선 후 점수: 0.134
  개선율: 0.0%

질문: 아로마타제억제제가 무엇인가요?
  개선

In [14]:
# 실험 결과를 표로 정리
import pandas as pd


def visualize_experiment_results(experiment_results):
    """
    실험 결과를 보기 좋게 정리
    """
    df = pd.DataFrame(experiment_results)

    # 피벗 테이블로 변환
    pivot = df.pivot_table(values="avg_score", index="chunk_size", columns="overlap")

    print("\n검색 품질 점수 (낮을수록 좋음):")
    print(pivot.round(3))

    # 최적 조합 하이라이트
    best_idx = df["avg_score"].idxmin()
    best_row = df.loc[best_idx]

    print(
        f"\n★ 최적 조합: chunk_size={best_row['chunk_size']}, "
        f"overlap={best_row['overlap']}"
    )
    print(f"   - 검색 점수: {best_row['avg_score']:.3f}")
    print(f"   - 청크 개수: {best_row['num_chunks']}")

    return best_row


# 실행
best_config = visualize_experiment_results(results)


검색 품질 점수 (낮을수록 좋음):
overlap       50     100    200
chunk_size                     
500         0.397  0.394  0.396
1000        0.416  0.413  0.414
1500        0.441  0.441  0.441

★ 최적 조합: chunk_size=500.0, overlap=100.0
   - 검색 점수: 0.394
   - 청크 개수: 21.0
